In [14]:
# Importing Libraries and setting up dir locations
import pandas as pd
import numpy as np
import re

data_loc = "data\\"

In [26]:
# Reading the Datasets

train_data_raw = pd.read_csv(data_loc + "train.csv")
test_data = pd.read_csv(data_loc + "test.csv")
misconcept = pd.read_csv(data_loc + "misconception_mapping.csv")

In [27]:
# Creating ID Text Mappings for later use

construct_map = train_data_raw[['ConstructId','ConstructName']].drop_duplicates()
subject_map = train_data_raw[['SubjectId','SubjectName']].drop_duplicates()

In [44]:
# Dropping the unecessary text fields which have been mapped
train_data = train_data_raw.drop(['ConstructName','SubjectName'], axis=1)



### Melting the dataframe to map each option with their Option ID
train_data = train_data.melt(id_vars = ['SubjectId','ConstructId','QuestionId','QuestionText','CorrectAnswer'],
                             var_name="Column Name", value_name="Value")
# using regex to get the Option ID A B C D
train_data['Option'] = train_data['Column Name'].apply(lambda x: re.sub("[^A-Z]","",x)[1])
# Using the Option ID to get the type of Column -> Answer Text or Misconception ID
train_data['Column Type'] = train_data.apply(lambda x: x['Column Name'][0] + x['Column Name'][1:].split(x['Option'])[0] + "Id", axis=1)
train_data = train_data.drop('Column Name', axis=1)


### Removing Lines where the answer is correct
train_data = train_data[train_data.apply(lambda x: x['CorrectAnswer']!=x['Option'], axis=1)]
# Pivoting back with 2 columns -> Answer Text and the Misonception
train_data = train_data.pivot(index = ['SubjectId','ConstructId','QuestionId','QuestionText','CorrectAnswer','Option'],
                 columns = 'Column Type', values= 'Value').reset_index()
# Removing Lines which dont have any misconception
no_misconcept = train_data[train_data['MisconceptionId'].isnull()!=False]
train_data = train_data[train_data['MisconceptionId'].isnull()==False]



### Getting the Misconception Text based on ID for my review
train_data['MisconceptionId'] = train_data['MisconceptionId'].astype(int)
misconcept['MisconceptionId'] = misconcept['MisconceptionId'].astype(int)
train_data = train_data.merge(misconcept.drop_duplicates(subset = 'MisconceptionId'), 
                              on = 'MisconceptionId', how = 'left', indicator = True)
print(train_data._merge.value_counts())
train_data = train_data.drop("_merge", axis=1)


_merge
both          4370
left_only        0
right_only       0
Name: count, dtype: int64


In [45]:
train_data.groupby("SubjectId")['MisconceptionId'].nunique().sort_values(ascending = False).value_counts()

MisconceptionId
3     10
8      9
9      9
2      8
5      8
11     8
12     8
14     8
1      7
7      7
17     6
4      6
10     6
15     6
19     5
28     4
22     4
21     4
20     4
6      4
18     4
13     4
16     3
35     2
34     2
32     2
23     2
25     2
24     2
29     1
30     1
26     1
47     1
36     1
37     1
42     1
45     1
49     1
Name: count, dtype: int64